# 32강 스케쥴러

- 스케쥴러
  - 특정 시간이나 정해진 간격에 따라어떤 함수가 자동으로 실행되는 처리
    - ex) 1시간 마다 한번씩 자동으로 수행되는 기능, 매일 자정에 자동으로 수행되는 기능

- 'apscheduler'가 파이썬 공식문서에서 소개되고 있으므로 권장
  - pip install apscheduler

### 01. 작업 준비

- 패키지 참조

In [6]:
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.cron import CronTrigger
from apscheduler.jobstores.base import JobLookupError
import datetime as dt
import time

- 스케쥴에 따라 자동으로 실행될 기능

In [2]:
def myjob(name):
    currentTime = dt.datetime.now()
    timeFormat = currentTime.strftime("%Y/%m/%d %H:%M:%S")
    print(f"[{name}] I'm working... | {timeFormat}")

In [3]:
myjob("lee")

[lee] I'm working... | 2023/12/01 11:07:44


### 02. 스케쥴러 등록

- 정해진 간격마다 실행하기
  - 매 3초마다 실행

In [4]:
sched = BackgroundScheduler()
sched.start()

sched.add_job(myjob, "interval", seconds = 3 ,args = ["kim"], id = "myjob1")
# 파라미터
# 1번째 파라미터 : 작업을 수행할 함수 명
# 2번째 파라미터 : 스케쥴러를 사용하는 방식, interval은 일정 간격마다
# 3번째 파라미터 : 방식에 대한 시간의 단위
# args : 수행할 작업의 파라미터
# id : 작업의 고유 아이디 지정, 지정하지 않으면 꽤 긴 문자열이 되므로 지정 추천
# 함수앞에 ?를 붙이면 파라미터 설명 출력 가능

<Job (id=myjob1 name=myjob)>

[kim] I'm working... | 2023/12/01 11:08:39
[kim] I'm working... | 2023/12/01 11:08:42


- interval로 사용할 수 있는 파라미터
  - 시간 관련 : seconds, minute, hour, day, month, year, week, day_of_week, start_date, end_date 등을 설정 가능
    - start_date와 end_date는 datetime 객체로 설정하며, 작업을 수행할 날짜 범위 지정

- 스케쥴러에서 작업 제거

In [5]:
sched.remove_job("myjob1")
# 백그라운드에서 스케쥴러 작업을 중단시키는 코드
# 패러미터로 해당 작업의 id를 지정

- cron 표현식으로 설정하기
  - cron 표현식
    - Linux, Mac 등에서 작업 스케쥴러를 등록할 때 사용하는 시간 단위 설정 표현식

In [ ]:
# 공백으로 구분하는 7자리의 값으로 구성됨

# ```shell
# *******
# ```

# 각 자리는 순서대로 '초, 분, 시, 일, 월, 요일, 년'을 의미함

- 값의 설정 방법

| 필드 | 허용되는 값 | 허용되는 특수문자 |
|---|---|---|
| 초(Seconds) | 0~59 | `,` `-` `*` `/` |
| 분(Minuets) | 0~59 | `,` `-` `*` `/` |
| 시(Hours) | 0~23 | `,` `-` `*` `/` |
| 일(Day of month) | 0~31 | `,` `-` `*` `/` `L` `W` |
| 월(Month) | 1~ 12 또는 JAN ~ DEC | `,` `-` `*` `/` |
| 요일(Day of week) | 0~6 또는 SUN ~ SAT | `,` `-` `*` `/` `L` `#` |
| 년(Year) | 0~59 | `,` `-` `*` `/` |

- 특수문자의 의미
  - `*` : 모든 값
  - `?` : 특정 값이 없음을 의미
  - `-` : 범위를 의미 (MON-WED는 월요일부터 수요일까지)
  - `,` : 특별한 값일때만 동작 (월, 금)
  - `/` : 시작시간 / 단위 (분에서 0/5 = 0분부터 매 5분)
  - `L` : 일에서 시작하면 마지막 일, 요일에서는 마지막 요일(토요일)
  - `W` : 가장 가까운 평일 (15W는 15일에서 가장 가까운 평일 (월 ~ 금)을 찾음)
  - `#` : 몇째 주의  무슨 요일을 표현 (3#2 2번째 주 수요일)

> crontab.cronhub.io

In [36]:
sched = BackgroundScheduler()
sched.start()

In [37]:
# 2 * * * * * * -> 매분 2초
# 매초마다 2초 간격으로

myTrigger = CronTrigger.from_crontab("* * * * *")
sched.add_job(myjob, myTrigger, args = ["kim"], id = "myjob2")

<Job (id=myjob2 name=myjob)>

[kim] I'm working... | 2023/12/01 11:37:00
[kim] I'm working... | 2023/12/01 11:38:00
[kim] I'm working... | 2023/12/01 11:39:00


In [38]:
sched.remove_job("myjob2")

In [ ]:
# 스케쥴러 종료
sched.shutdown()

- 특정 시각에 수행

In [ ]:
# 정해진 시각에 1회만 처리하기
targetDate = dt.datetime(2023, 12, 1, 11, 43, 0)
sched.add_job(myjob, 'date', run_date = targetDate, args = ['park'], id = 'myjob4')

- 메일링 리스트 개선

In [39]:
# 앞 단원에서 구현한 메일 발송 모듈
import MyMailer

# 비동기 처리 기능을 제공하는 모듈
import concurrent.futures as futures

In [40]:
# 메일링 리스트 개선
today = dt.datetime.now()
year = today.year
month = today.month
day = today.day

fromAddr = "yijingue@naver.com"
subjectTmpl = "{name}님의 {yy}년 {mm}월 급여명세서 입니다."

with open('mailtest/content.txt', 'r', encoding = 'utf-8') as f:
    contentTmpl = f.read()

In [41]:
# 비동기식 메일 발송
def sendmail():
    startTime = dt.datetime.now()

    with open("mailtest/mail_list.csv", "r", encoding = 'euc-kr') as f:
        csv = f.readlines()

        with futures.ThreadPoolExecutor(max_workers = 10) as executor:
            for line in csv:
                name, email, file1, file2 = line.strip().split(",")
                toAddr = "{email}".format(email = email)
                subject = subjectTmpl.format(name = name, yy = year, mm = month)
                content = contentTmpl.format(name = name, yy = year, mm = month, dd = day)
            
                executor.submit(MyMailer.sendMail, fromAddr, toAddr, subject, content, [file1, file2])

    endTime = dt.datetime.now()
    workTime = endTime - startTime
    print(f"작업에 소요된 시간은 총 {workTime.seconds}초 입니다.")

In [42]:
sched = BackgroundScheduler()
sched.start()

In [43]:
sched.add_job(sendmail, 'cron', second = '*/5', id = 'mail1')

<Job (id=mail1 name=sendmail)>

작업에 소요된 시간은 총 2초 입니다.
작업에 소요된 시간은 총 0초 입니다.
작업에 소요된 시간은 총 1초 입니다.
작업에 소요된 시간은 총 0초 입니다.
작업에 소요된 시간은 총 0초 입니다.


In [45]:
try:
    sched.remove_job('mail1')
    sched.shutdown()
except JobLookupError as je:
    print("스케쥴러 중지에 실패했습니다.", je)

스케쥴러 중지에 실패했습니다. 'No job by the id of mail1 was found'
